In [1]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
!conda install -c conda-forge geocoder --yes 
import geocoder
!conda install -c conda-forge wikipedia --yes
import wikipedia as wp
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header = 0)[0]

df.head(20)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [3]:
df_C = df[df.Borough != 'Not assigned']
df_C.reset_index(drop = True, inplace = True)

df_C.rename(columns = {'Neighbourhood' : 'Neighborhood'}, inplace = True)

df_C.head(30)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [4]:
df_C = df_C.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(set(x)))

df_C.head(30)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
na_nb_idx = df_C['Neighborhood'] == 'Not assigned'
df_C.at[na_nb_idx, 'Neighborhood'] = df_C[na_nb_idx].Borough

In [6]:
df_C.head(30)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
df_C = df_C[df_C['Borough'].str.contains('Toronto')]
df_C.reset_index(drop = True, inplace = True)

df_C.head(30)

,Postcode,Borough,Neighborhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"Riverdale, The Danforth West"
2,M4L,East Toronto,"India Bazaar, The Beaches West"
3,M4M,East Toronto,Studio District
4,M4N,Central Toronto,Lawrence Park
5,M4P,Central Toronto,Davisville North
6,M4R,Central Toronto,North Toronto West
7,M4S,Central Toronto,Davisville
8,M4T,Central Toronto,"Summerhill East, Moore Park"
9,M4V,Central Toronto,"Summerhill West, Deer Park, Forest Hill SE, Ra..."


In [8]:
df_C.drop_duplicates(subset = ['Neighborhood'], inplace = True)

df_C.head(30)

,Postcode,Borough,Neighborhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"Riverdale, The Danforth West"
2,M4L,East Toronto,"India Bazaar, The Beaches West"
3,M4M,East Toronto,Studio District
4,M4N,Central Toronto,Lawrence Park
5,M4P,Central Toronto,Davisville North
6,M4R,Central Toronto,North Toronto West
7,M4S,Central Toronto,Davisville
8,M4T,Central Toronto,"Summerhill East, Moore Park"
9,M4V,Central Toronto,"Summerhill West, Deer Park, Forest Hill SE, Ra..."


In [9]:
df_C['Latitude'] = np.nan
df_C['Longitude'] = np.nan

for idx, nb in zip(df_C.index, df_C['Neighborhood']):
    geolocator = Nominatim()
    try:
        g = geolocator.geocode('{}, Toronto, Ontario'.format(nb))
        df_C.at[idx, 'Latitude'] = g.latitude
        df_C.at[idx, 'Longitude'] = g.longitude
    except:
        df_C.at[idx, 'Latitude'] = 0
        df_C.at[idx, 'Longitude'] = 0

df_C = df_C[df_C['Latitude'] != 0]

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [10]:
df_C

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.671024,-79.296712
1,M4K,East Toronto,"Riverdale, The Danforth West",43.665470,-79.352594
3,M4M,East Toronto,Studio District,43.644395,-79.399570
4,M4N,Central Toronto,Lawrence Park,43.729199,-79.403252
5,M4P,Central Toronto,Davisville North,43.704312,-79.388517
6,M4R,Central Toronto,North Toronto West,43.646547,-79.419526
7,M4S,Central Toronto,Davisville,43.704312,-79.388517
8,M4T,Central Toronto,"Summerhill East, Moore Park",43.681678,-79.390504
10,M4W,Downtown Toronto,Rosedale,43.678356,-79.380746
11,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.664087,-79.370715


In [42]:
CLIENT_ID = 'NNCJQ0BEUM5CGFEPHYXPA2Q5RFNNB5H1CSFCQYFQN3QZF4OS'
CLIENT_SECRET = 'VLUGOTBAYHV0SCQNJFV1FDER014J3HCSHUS341UA2DLTPDNC'
VERSION = '20191001'

In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500): 
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        jres = requests.get(url).json()
        filename = name + '.json'
        with open(filename, 'w') as outfile:
             json.dump(jres, outfile)        
        
        results = jres["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
LIMIT = 100000
neighborhood_venues = getNearbyVenues(df_C['Neighborhood'], df_C['Latitude'], df_C['Longitude'], 500)
neighborhood_venues.shape

The Beaches
Riverdale, The Danforth West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East, Moore Park
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
St. James Town
Berczy Park
Central Bay Street
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Christie
Dufferin, Dovercourt Village
Roncesvalles, Parkdale
Runnymede, Swansea


(1243, 7)

In [47]:
neighborhood_venues.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.671024,-79.296712,Kew Gardens,43.669038,-79.298538,Park
1,The Beaches,43.671024,-79.296712,Mastermind Toys,43.671453,-79.293971,Toy / Game Store
2,The Beaches,43.671024,-79.296712,The Ten Spot,43.670340,-79.299363,Nail Salon
3,The Beaches,43.671024,-79.296712,Sanna's Farmacia,43.670929,-79.295969,Juice Bar
4,The Beaches,43.671024,-79.296712,Kew-Balmy Beach,43.667372,-79.295312,Beach
5,The Beaches,43.671024,-79.296712,Castro's Lounge,43.671104,-79.295107,Bar
6,The Beaches,43.671024,-79.296712,DAVIDsTEA,43.670169,-79.299498,Tea Room
7,The Beaches,43.671024,-79.296712,Yumei Sushi,43.671108,-79.295064,Japanese Restaurant
8,The Beaches,43.671024,-79.296712,Pippins Tea Company,43.670992,-79.295905,Tea Room
9,The Beaches,43.671024,-79.296712,Sunset Grill,43.670214,-79.299653,Diner


In [48]:
len(neighborhood_venues['Venue Category'].unique())

208

In [49]:
list(neighborhood_venues['Venue Category'].unique())

['Park',
 'Toy / Game Store',
 'Nail Salon',
 'Juice Bar',
 'Beach',
 'Bar',
 'Tea Room',
 'Japanese Restaurant',
 'Diner',
 'BBQ Joint',
 'Liquor Store',
 'Grocery Store',
 'Bakery',
 'Bookstore',
 'Breakfast Spot',
 'Jazz Club',
 'Sandwich Place',
 'Pharmacy',
 'Thai Restaurant',
 'Chocolate Shop',
 'Greek Restaurant',
 'Pub',
 'Pizza Place',
 'Café',
 'Coffee Shop',
 'Bank',
 'Burger Joint',
 'Mediterranean Restaurant',
 'Chinese Restaurant',
 'Tree',
 'Restaurant',
 'Salon / Barbershop',
 'Jewelry Store',
 'Shopping Mall',
 'Martial Arts Dojo',
 'Cheese Shop',
 'Portuguese Restaurant',
 'Skating Rink',
 'Asian Restaurant',
 'Ice Cream Shop',
 'Vietnamese Restaurant',
 'Vegetarian / Vegan Restaurant',
 'French Restaurant',
 'Neighborhood',
 'Fast Food Restaurant',
 'Dim Sum Restaurant',
 'Light Rail Station',
 'Fish Market',
 'Gym / Fitness Center',
 'Trail',
 'Baseball Field',
 'Mexican Restaurant',
 'Italian Restaurant',
 'Spa',
 'Caribbean Restaurant',
 'Steakhouse',
 'Beer Bar',

In [50]:
neighborhood_venues_L = neighborhood_venues[neighborhood_venues['Venue Category'].str.contains('Market|Asian|Supermarket|Pastry Shop|Gourmet Shop|Food & Drink Shop|Dessert Shop|Vietnamese|Japanese|Thai|Chinese|Deli|Malay|Ramen|Bus|Bakery|Grocery|Liquor')]

In [51]:
neighborhood_venues_L.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
7,The Beaches,43.671024,-79.296712,Yumei Sushi,43.671108,-79.295064,Japanese Restaurant
12,The Beaches,43.671024,-79.296712,LCBO,43.669993,-79.300383,Liquor Store
13,The Beaches,43.671024,-79.296712,Foodland - Toronto,43.670736,-79.298267,Grocery Store
14,The Beaches,43.671024,-79.296712,COBS Bread,43.669881,-79.300828,Bakery
21,The Beaches,43.671024,-79.296712,Green Basil,43.671200,-79.294717,Thai Restaurant
26,The Beaches,43.671024,-79.296712,Bikkuri Japanese Restaurant,43.670715,-79.296956,Japanese Restaurant
34,The Beaches,43.671024,-79.296712,Honey Bee Restaurant,43.670409,-79.298406,Chinese Restaurant
43,The Beaches,43.671024,-79.296712,Brick Street Bakery,43.669694,-79.301530,Bakery
47,The Beaches,43.671024,-79.296712,The Thai Grill,43.669585,-79.302048,Thai Restaurant
49,"Riverdale, The Danforth West",43.665470,-79.352594,Ha Noi 3 Seasons,43.665578,-79.352153,Asian Restaurant


In [52]:
neighborhood_venues_L.reset_index(drop = True, inplace = True)

neighborhood_venues_L.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.671024,-79.296712,Yumei Sushi,43.671108,-79.295064,Japanese Restaurant
1,The Beaches,43.671024,-79.296712,LCBO,43.669993,-79.300383,Liquor Store
2,The Beaches,43.671024,-79.296712,Foodland - Toronto,43.670736,-79.298267,Grocery Store
3,The Beaches,43.671024,-79.296712,COBS Bread,43.669881,-79.300828,Bakery
4,The Beaches,43.671024,-79.296712,Green Basil,43.671200,-79.294717,Thai Restaurant
5,The Beaches,43.671024,-79.296712,Bikkuri Japanese Restaurant,43.670715,-79.296956,Japanese Restaurant
6,The Beaches,43.671024,-79.296712,Honey Bee Restaurant,43.670409,-79.298406,Chinese Restaurant
7,The Beaches,43.671024,-79.296712,Brick Street Bakery,43.669694,-79.301530,Bakery
8,The Beaches,43.671024,-79.296712,The Thai Grill,43.669585,-79.302048,Thai Restaurant
9,"Riverdale, The Danforth West",43.665470,-79.352594,Ha Noi 3 Seasons,43.665578,-79.352153,Asian Restaurant


In [53]:
neighborhood_onehot = pd.get_dummies(neighborhood_venues_L[['Venue Category']], prefix="", prefix_sep="")

neighborhood_onehot['Neighborhood'] = neighborhood_venues_L['Neighborhood']

cols = neighborhood_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
neighborhood_onehot = neighborhood_onehot.reindex(columns= cols)

neighborhood_onehot.head(30)

,Neighborhood,Asian Restaurant,Bakery,Bus Line,Chinese Restaurant,Deli / Bodega,Dessert Shop,Farmers Market,Fish Market,Food & Drink Shop,Gourmet Shop,Grocery Store,Japanese Restaurant,Liquor Store,Malay Restaurant,Market,Ramen Restaurant,Thai Restaurant,Vietnamese Restaurant
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,The Beaches,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,The Beaches,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
6,The Beaches,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,The Beaches,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,"Riverdale, The Danforth West",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()

In [55]:
neighborhood_grouped.head(30)

,Neighborhood,Asian Restaurant,Bakery,Bus Line,Chinese Restaurant,Deli / Bodega,Dessert Shop,Farmers Market,Fish Market,Food & Drink Shop,Gourmet Shop,Grocery Store,Japanese Restaurant,Liquor Store,Malay Restaurant,Market,Ramen Restaurant,Thai Restaurant,Vietnamese Restaurant
0,Berczy Park,0.000000,0.230769,0.000,0.000000,0.076923,0.000000,0.153846,0.076923,0.000000,0.076923,0.076923,0.230769,0.000000,0.0,0.000000,0.000000,0.076923,0.000000
1,"Cabbagetown, St. James Town",0.000000,0.166667,0.000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.166667,0.083333,0.0,0.083333,0.000000,0.166667,0.083333
2,Central Bay Street,0.000000,0.250000,0.000,0.250000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.0,0.000000,0.125000,0.125000,0.000000
3,Christie,0.000000,0.000000,0.000,0.111111,0.000000,0.222222,0.000000,0.000000,0.000000,0.000000,0.222222,0.222222,0.000000,0.0,0.000000,0.111111,0.000000,0.111111
4,Church and Wellesley,0.000000,0.000000,0.000,0.090909,0.000000,0.090909,0.000000,0.000000,0.090909,0.000000,0.090909,0.363636,0.000000,0.0,0.000000,0.181818,0.090909,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.181818,0.000,0.000000,0.272727,0.000000,0.181818,0.000000,0.000000,0.000000,0.000000,0.272727,0.000000,0.0,0.000000,0.000000,0.090909,0.000000
6,Davisville,0.000000,0.000000,0.000,0.000000,0.142857,0.428571,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.142857,0.000000
7,Davisville North,0.000000,0.000000,0.000,0.000000,0.142857,0.428571,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.142857,0.000000
8,"Design Exchange, Toronto Dominion Centre",0.100000,0.300000,0.000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.0,0.000000,0.000000,0.200000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.500000,0.000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.125000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.125000


In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common'.format(ind+1))

neighborhood_sorted = pd.DataFrame(columns=columns)
neighborhood_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhood_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhood_sorted

,Neighborhood,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common,6th Most Common,7th Most Common,8th Most Common,9th Most Common,10th Most Common
0,Berczy Park,Bakery,Japanese Restaurant,Farmers Market,Deli / Bodega,Grocery Store,Gourmet Shop,Thai Restaurant,Fish Market,Vietnamese Restaurant,Dessert Shop
1,"Cabbagetown, St. James Town",Bakery,Japanese Restaurant,Thai Restaurant,Vietnamese Restaurant,Market,Liquor Store,Grocery Store,Chinese Restaurant,Deli / Bodega,Dessert Shop
2,Central Bay Street,Bakery,Chinese Restaurant,Ramen Restaurant,Japanese Restaurant,Dessert Shop,Thai Restaurant,Vietnamese Restaurant,Farmers Market,Bus Line,Deli / Bodega
3,Christie,Japanese Restaurant,Grocery Store,Dessert Shop,Vietnamese Restaurant,Ramen Restaurant,Chinese Restaurant,Farmers Market,Bakery,Bus Line,Deli / Bodega
4,Church and Wellesley,Japanese Restaurant,Ramen Restaurant,Food & Drink Shop,Chinese Restaurant,Grocery Store,Thai Restaurant,Dessert Shop,Farmers Market,Bakery,Bus Line
5,"Commerce Court, Victoria Hotel",Deli / Bodega,Japanese Restaurant,Bakery,Farmers Market,Thai Restaurant,Vietnamese Restaurant,Fish Market,Bus Line,Chinese Restaurant,Dessert Shop
6,Davisville,Dessert Shop,Deli / Bodega,Japanese Restaurant,Farmers Market,Thai Restaurant,Vietnamese Restaurant,Fish Market,Bakery,Bus Line,Chinese Restaurant
7,Davisville North,Dessert Shop,Deli / Bodega,Japanese Restaurant,Farmers Market,Thai Restaurant,Vietnamese Restaurant,Fish Market,Bakery,Bus Line,Chinese Restaurant
8,"Design Exchange, Toronto Dominion Centre",Bakery,Deli / Bodega,Thai Restaurant,Asian Restaurant,Japanese Restaurant,Fish Market,Bus Line,Chinese Restaurant,Dessert Shop,Farmers Market
9,"Dufferin, Dovercourt Village",Bakery,Vietnamese Restaurant,Japanese Restaurant,Grocery Store,Farmers Market,Fish Market,Bus Line,Chinese Restaurant,Deli / Bodega,Dessert Shop


In [58]:
kclusters = 5

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

kmeans.labels_

array([0, 2, 2, 1, 1, 0, 4, 4, 0, 0, 0, 0, 2, 2, 3, 2, 2, 1, 2, 2, 2],
      dtype=int32)

In [59]:
neighborhood_merged = neighborhood_sorted

neighborhood_merged['Cluster Labels'] = kmeans.labels_

neighborhood_merged['Latitude'] = np.nan
neighborhood_merged['Longitude'] = np.nan

for idx in neighborhood_merged.index.values:
    cord_idx = df_C['Neighborhood'] == neighborhood_merged.loc[idx, 'Neighborhood']
    neighborhood_merged.at[idx, 'Latitude'] = df_C.loc[cord_idx, 'Latitude'].values
    neighborhood_merged.at[idx, 'Longitude'] = df_C.loc[cord_idx, 'Longitude'].values

neighborhood_merged.head()

,Neighborhood,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common,6th Most Common,7th Most Common,8th Most Common,9th Most Common,10th Most Common,Cluster Labels,Latitude,Longitude
0,Berczy Park,Bakery,Japanese Restaurant,Farmers Market,Deli / Bodega,Grocery Store,Gourmet Shop,Thai Restaurant,Fish Market,Vietnamese Restaurant,Dessert Shop,0,43.647984,-79.375396
1,"Cabbagetown, St. James Town",Bakery,Japanese Restaurant,Thai Restaurant,Vietnamese Restaurant,Market,Liquor Store,Grocery Store,Chinese Restaurant,Deli / Bodega,Dessert Shop,2,43.664087,-79.370715
2,Central Bay Street,Bakery,Chinese Restaurant,Ramen Restaurant,Japanese Restaurant,Dessert Shop,Thai Restaurant,Vietnamese Restaurant,Farmers Market,Bus Line,Deli / Bodega,2,43.656962,-79.384224
3,Christie,Japanese Restaurant,Grocery Store,Dessert Shop,Vietnamese Restaurant,Ramen Restaurant,Chinese Restaurant,Farmers Market,Bakery,Bus Line,Deli / Bodega,1,43.664111,-79.418405
4,Church and Wellesley,Japanese Restaurant,Ramen Restaurant,Food & Drink Shop,Chinese Restaurant,Grocery Store,Thai Restaurant,Dessert Shop,Farmers Market,Bakery,Bus Line,1,43.665524,-79.383801


In [60]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [61]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_merged['Latitude'], neighborhood_merged['Longitude'], neighborhood_merged['Neighborhood'], neighborhood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

In [62]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 0, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common,6th Most Common,7th Most Common,8th Most Common,9th Most Common
0,Berczy Park,0,Bakery,Japanese Restaurant,Farmers Market,Deli / Bodega,Grocery Store,Gourmet Shop,Thai Restaurant,Fish Market,Vietnamese Restaurant
5,"Commerce Court, Victoria Hotel",0,Deli / Bodega,Japanese Restaurant,Bakery,Farmers Market,Thai Restaurant,Vietnamese Restaurant,Fish Market,Bus Line,Chinese Restaurant
8,"Design Exchange, Toronto Dominion Centre",0,Bakery,Deli / Bodega,Thai Restaurant,Asian Restaurant,Japanese Restaurant,Fish Market,Bus Line,Chinese Restaurant,Dessert Shop
9,"Dufferin, Dovercourt Village",0,Bakery,Vietnamese Restaurant,Japanese Restaurant,Grocery Store,Farmers Market,Fish Market,Bus Line,Chinese Restaurant,Deli / Bodega
10,Lawrence Park,0,Bakery,Asian Restaurant,Bus Line,Deli / Bodega,Japanese Restaurant,Fish Market,Chinese Restaurant,Dessert Shop,Farmers Market
11,North Toronto West,0,Asian Restaurant,Bakery,Vietnamese Restaurant,Malay Restaurant,Japanese Restaurant,Deli / Bodega,Market,Liquor Store,Grocery Store


In [63]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 1, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common,6th Most Common,7th Most Common,8th Most Common,9th Most Common
3,Christie,1,Japanese Restaurant,Grocery Store,Dessert Shop,Vietnamese Restaurant,Ramen Restaurant,Chinese Restaurant,Farmers Market,Bakery,Bus Line
4,Church and Wellesley,1,Japanese Restaurant,Ramen Restaurant,Food & Drink Shop,Chinese Restaurant,Grocery Store,Thai Restaurant,Dessert Shop,Farmers Market,Bakery
17,St. James Town,1,Grocery Store,Food & Drink Shop,Bakery,Market,Fish Market,Bus Line,Chinese Restaurant,Deli / Bodega,Dessert Shop


In [64]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 2, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common,6th Most Common,7th Most Common,8th Most Common,9th Most Common
1,"Cabbagetown, St. James Town",2,Bakery,Japanese Restaurant,Thai Restaurant,Vietnamese Restaurant,Market,Liquor Store,Grocery Store,Chinese Restaurant,Deli / Bodega
2,Central Bay Street,2,Bakery,Chinese Restaurant,Ramen Restaurant,Japanese Restaurant,Dessert Shop,Thai Restaurant,Vietnamese Restaurant,Farmers Market,Bus Line
12,"Riverdale, The Danforth West",2,Vietnamese Restaurant,Bakery,Chinese Restaurant,Grocery Store,Fish Market,Asian Restaurant,Japanese Restaurant,Liquor Store,Gourmet Shop
13,"Roncesvalles, Parkdale",2,Gourmet Shop,Bakery,Liquor Store,Dessert Shop,Thai Restaurant,Fish Market,Vietnamese Restaurant,Farmers Market,Bus Line
15,Roselawn,2,Bakery,Vietnamese Restaurant,Grocery Store,Deli / Bodega,Thai Restaurant,Food & Drink Shop,Ramen Restaurant,Japanese Restaurant,Liquor Store
16,"Runnymede, Swansea",2,Bakery,Liquor Store,Dessert Shop,Gourmet Shop,Thai Restaurant,Vietnamese Restaurant,Farmers Market,Bus Line,Chinese Restaurant
18,Studio District,2,Asian Restaurant,Bakery,Dessert Shop,Liquor Store,Thai Restaurant,Food & Drink Shop,Market,Japanese Restaurant,Grocery Store
19,"Summerhill East, Moore Park",2,Asian Restaurant,Bakery,Liquor Store,Grocery Store,Gourmet Shop,Thai Restaurant,Dessert Shop,Farmers Market,Bus Line
20,The Beaches,2,Bakery,Japanese Restaurant,Thai Restaurant,Chinese Restaurant,Liquor Store,Grocery Store,Vietnamese Restaurant,Farmers Market,Bus Line


In [65]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 3, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common,6th Most Common,7th Most Common,8th Most Common,9th Most Common
14,Rosedale,3,Japanese Restaurant,Vietnamese Restaurant,Fish Market,Bakery,Bus Line,Chinese Restaurant,Deli / Bodega,Dessert Shop,Farmers Market


In [66]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 4, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common,6th Most Common,7th Most Common,8th Most Common,9th Most Common
6,Davisville,4,Dessert Shop,Deli / Bodega,Japanese Restaurant,Farmers Market,Thai Restaurant,Vietnamese Restaurant,Fish Market,Bakery,Bus Line
7,Davisville North,4,Dessert Shop,Deli / Bodega,Japanese Restaurant,Farmers Market,Thai Restaurant,Vietnamese Restaurant,Fish Market,Bakery,Bus Line
